<a href="https://colab.research.google.com/github/galdir/classificacao_com_bert/blob/main/classificacao_de_textos_com_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classificação de Textos usando BERT

# Instalação das bibliotecas necessárias


In [1]:
# Descomente e execute se necessário
#!pip install transformers pandas numpy scikit-learn torch datasets

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

# Importação das bibliotecas


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Verificar se GPU está disponível


In [4]:
dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando {dispositivo}")

Usando cuda


# Baixar e preparar o dataset B2W-Reviews01

https://github.com/americanas-tech/b2w-reviews01/tree/main

In [5]:
print("Baixando o dataset B2W-Reviews01...")
url = "https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv"
df = pd.read_csv(url, low_memory=False)

Baixando o dataset B2W-Reviews01...


In [6]:
df.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


# Preparar os dados - apenas textos e sentimentos


In [7]:
df = df[['review_text', 'overall_rating']]
df.columns = ['texto', 'nota']

In [8]:
df.head(10)

,texto,nota
0,Estou contente com a compra entrega rápida o ú...,4
1,"Por apenas R$1994.20,eu consegui comprar esse ...",4
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,4
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,4
4,"A entrega foi no prazo, as americanas estão de...",5
5,"Excelente produto, por fora em material acríli...",5
6,"produto mto bom, com essa garrafinha vc pode a...",5
7,Produto excelente qualidade boa câmera desenvo...,4
8,O barulho e minimo e o vento é bem forte na ve...,5
9,MEU PRODUTO NAO FOI ENTREGUE E A AMERICANAS ES...,1


## Converter notas para sentimento binário (0-negativo, 1-positivo)


In [9]:
df = df[(df['nota'] <= 2) | (df['nota'] >= 4)].copy()

In [10]:
df['sentimento'] = df['nota'].apply(lambda x: 1 if x >= 4 else 0)

In [11]:
df = df[['texto', 'sentimento']]

In [12]:
df = df[df['texto'].notna() & (df['texto'] != '')].reset_index(drop=True)

In [13]:
df.head(10)

,texto,sentimento
0,Estou contente com a compra entrega rápida o ú...,1
1,"Por apenas R$1994.20,eu consegui comprar esse ...",1
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1
4,"A entrega foi no prazo, as americanas estão de...",1
5,"Excelente produto, por fora em material acríli...",1
6,"produto mto bom, com essa garrafinha vc pode a...",1
7,Produto excelente qualidade boa câmera desenvo...,1
8,O barulho e minimo e o vento é bem forte na ve...,1
9,MEU PRODUTO NAO FOI ENTREGUE E A AMERICANAS ES...,0


## Verificar distribuição


In [14]:
print(f"Total de exemplos: {len(df)}")
print(f"Exemplos positivos: {sum(df['sentimento'])}")
print(f"Exemplos negativos: {len(df) - sum(df['sentimento'])}")

Total de exemplos: 113088
Exemplos positivos: 79316
Exemplos negativos: 33772


## Limitar dataset para processamento mais rápido


In [15]:
# Usando apenas 1000 exemplos balanceados para demonstração
neg = df[df['sentimento'] == 0].sample(500, random_state=42)
pos = df[df['sentimento'] == 1].sample(500, random_state=42)
df = pd.concat([neg, pos]).sample(frac=1, random_state=42).reset_index(drop=True)

In [16]:
df.shape

(1000, 2)

## Separar em treino e teste

In [17]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


# Tokenização

## Carregar o Tokenizador do modelo BERT pré-treinado para português

In [18]:
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Função de pré-processamento para tokenização


In [19]:
exemplo = train_df.iloc[0]["texto"]
exemplo

'Recomendo, excelente produto, de fácil manuseio e ideal para quem tem pouco tempo!'

In [20]:
tokenizer(exemplo, truncation=True, padding="max_length", max_length=128)

{'input_ids': [101, 2325, 3309, 214, 117, 9235, 3576, 117, 125, 6904, 8473, 1556, 122, 7503, 221, 1977, 376, 1695, 596, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
def preprocess_function(examples):
    # Tokenizar os textos e definir truncation/padding
    tokenized = tokenizer(
        examples["texto"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # IMPORTANTE: Adicionar as etiquetas (labels)
    tokenized["labels"] = examples["sentimento"]

    return tokenized

In [22]:
preprocess_function(train_df.iloc[0])

{'input_ids': [101, 2325, 3309, 214, 117, 9235, 3576, 117, 125, 6904, 8473, 1556, 122, 7503, 221, 1977, 376, 1695, 596, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Converter DataFrames para o formato compatível com o Pytorch


In [23]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [24]:
train_dataset

Dataset({
    features: ['texto', 'sentimento', '__index_level_0__'],
    num_rows: 800
})

## Aplicar tokenização

In [25]:
train_tokenized = train_dataset.map(preprocess_function, batched=True)
test_tokenized = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [26]:
train_tokenized

Dataset({
    features: ['texto', 'sentimento', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 800
})

# Preparar o modelo para treinamento

## Carregar modelo pretreinado e preparado para classificação

In [27]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    device_map="auto"
)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Similaridade com embeddings do BERT

In [29]:
texto = "Este produto é excelente e durável."

# Tokenizar o texto
entradas = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Mover inputs para o mesmo dispositivo que o modelo
entradas = {k: v.to(dispositivo) for k, v in entradas.items()}

# Obter embeddings (representação vetorial) da frase
with torch.no_grad():
    outputs = model(**entradas, output_hidden_states=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [30]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0017,  0.0208]], device='cuda:0'), hidden_states=(tensor([[[-0.2793,  0.0076, -0.0980,  ...,  0.1093, -0.1878, -0.0169],
         [ 0.0705,  0.0779, -0.1107,  ...,  0.5663, -0.1148,  0.1654],
         [-0.2505, -0.2248,  0.5911,  ...,  1.0879,  1.2864,  0.3059],
         ...,
         [-0.2662, -0.0918,  0.5947,  ..., -0.1539,  0.6436,  0.1598],
         [-0.0351,  0.0996,  0.2141,  ..., -0.9158,  0.0897, -0.3697],
         [-0.0737,  0.6651,  0.3735,  ...,  0.1084,  0.5971,  0.1313]]],
       device='cuda:0'), tensor([[[ 1.2338e-02, -4.4660e-02,  2.1367e-02,  ...,  3.4905e-04,
          -4.7881e-02,  8.8600e-02],
         [-5.4083e-01,  2.6595e-02, -6.3600e-01,  ...,  7.2062e-01,
          -2.8861e-01, -2.0441e-01],
         [-3.8760e-01,  3.9820e-01,  4.6614e-01,  ...,  1.4548e+00,
           8.7859e-01, -8.6942e-04],
         ...,
         [ 5.7696e-02, -1.1834e-01,  2.1898e-01,  ...,  2.5846e-02,
           6.0091e-01, -7.3787e

In [31]:
outputs.keys()

odict_keys(['logits', 'hidden_states'])

logits é a saida de classificacão... que não treinamos ainda e não fazem sentido agora

In [32]:
outputs.logits

tensor([[-0.0017,  0.0208]], device='cuda:0')

hidden_states são as representações internas que o modelo constrói

In [33]:
outputs.hidden_states[-1].shape

torch.Size([1, 11, 768])

In [34]:
def obter_embedding(texto):
    # Tokenizar o texto
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

    # Mover inputs para o mesmo dispositivo que o modelo
    inputs = {k: v.to(dispositivo) for k, v in inputs.items()}

    # Obter embeddings (representação vetorial) da frase
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Usar a representação da camada de saída [CLS]
    # O token [CLS] (primeiro token) contém a representação da frase inteira
    embedding = outputs.hidden_states[-1][0, 0, :].cpu().numpy()
    return embedding

In [35]:
def calcular_similaridade(frase1, frase2):
    # Obter embeddings para cada frase
    emb1 = obter_embedding(frase1)
    emb2 = obter_embedding(frase2)

    # Normalizar os vetores
    emb1_norm = emb1 / np.linalg.norm(emb1)
    emb2_norm = emb2 / np.linalg.norm(emb2)

    # Calcular similaridade de cosseno
    similaridade = np.dot(emb1_norm, emb2_norm)
    return similaridade

In [36]:
frase1= "Este produto é excelente e durável."
frase2= "Este produto é ruim e ineficiente."

calcular_similaridade(frase1, frase2)

np.float32(0.8635869)

In [37]:
frase1= "Este produto é excelente e durável."
frase2= "O produto tem ótima qualidade e é resistente."

calcular_similaridade(frase1, frase2)

np.float32(0.96269107)

In [38]:
frase1= "Este produto é excelente e durável."
frase2= "Subi num ônibus em madagascar"

calcular_similaridade(frase1, frase2)

np.float32(0.5282244)

# Treinamento

## Configurar argumentos de treinamento


In [39]:
training_args = TrainingArguments(
    output_dir="./resultados",
    num_train_epochs=1,            # Apenas 1 época para demonstração
    #per_device_train_batch_size=8,
    #learning_rate=2e-5,
)

## Função para cálculo das métricas

In [40]:
def compute_metrics(resultado):
    rotulos_verdadeiros = resultado.label_ids
    rotulos_preditos = resultado.predictions.argmax(-1)
    acc = (rotulos_preditos == rotulos_verdadeiros).mean()
    return {"accuracy": acc}

## Configurar o treinador


In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [43]:
# import wandb
# wandb.init(mode="disabled")

## Treinar o modelo


In [44]:
print("\nIniciando treinamento...")
trainer.train()


Iniciando treinamento...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: galdir (ctai-ufba) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=100, training_loss=0.2515751075744629, metrics={'train_runtime': 46.8808, 'train_samples_per_second': 17.065, 'train_steps_per_second': 2.133, 'total_flos': 52622211072000.0, 'train_loss': 0.2515751075744629, 'epoch': 1.0})

# Avaliar o modelo


In [45]:
print("\nAvaliando o modelo...")
eval_results = trainer.evaluate()
print(f"Acurácia: {eval_results['eval_accuracy']:.4f}")


Avaliando o modelo...


Acurácia: 0.9400


## Função para classificar novos textos


In [46]:
def classificar_texto(texto):
    # Tokenizar e preparar o texto para o modelo
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

    # Mover inputs para o mesmo dispositivo que o modelo
    inputs = {k: v.to(dispositivo) for k, v in inputs.items()}

    # Fazer a predição
    with torch.no_grad():
        outputs = model(**inputs)

    # Obter a classe prevista (0=Negativo, 1=Positivo)
    sentimento = "Positivo" if outputs.logits.argmax().item() == 1 else "Negativo"

    return sentimento

## Testar com novos exemplos

In [47]:
texto = "Produto excelente, entrega rápida e atendimento nota 10!"

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: Produto excelente, entrega rápida e atendimento nota 10!
Classificação: Positivo


In [48]:
texto = "Péssima experiência, o produto veio quebrado e o suporte não resolveu."

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: Péssima experiência, o produto veio quebrado e o suporte não resolveu.
Classificação: Negativo


In [49]:
texto = "eu recomendaria"

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: eu recomendaria
Classificação: Positivo


In [50]:
texto = "eu desrecomendaria"

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: eu desrecomendaria
Classificação: Negativo


In [51]:
texto = "normal, não supreendeu."

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: normal, não supreendeu.
Classificação: Negativo
